In [ ]:
from google.colab import drive

In [ ]:
import pandas as pd

In [ ]:
#experimenting with pipelines
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle as pkl

In [ ]:
import sklearn
print(sklearn.__version__)

1.2.2


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data  = pd.read_csv('/content/drive/MyDrive/Datasets/male_players (legacy).csv')
data_copy = pd.read_csv('/content/drive/MyDrive/Datasets/male_players (legacy).csv')

<ipython-input-11-f5c7491910c1>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data  = pd.read_csv('/content/drive/MyDrive/Datasets/male_players (legacy).csv')
<ipython-input-11-f5c7491910c1>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data_copy = pd.read_csv('/content/drive/MyDrive/Datasets/male_players (legacy).csv')


In [ ]:
players_22 = pd.read_csv('/content/drive/MyDrive/Datasets/players_21.csv')

Feature engineering

In [ ]:
df = data
df.drop(columns = ['fifa_update_date', 'club_joined_date', 'player_id', 'fifa_version', 'league_id', 'club_contract_valid_until_year', 'fifa_update'], inplace= True)
#dropping useless columns
y = df['overall']


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 103 entries, player_url to player_face_url
dtypes: float64(16), int64(42), object(45)
memory usage: 127.0+ MB


In [ ]:
df.describe()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_team_id,club_jersey_number,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,161583.000000,161583.000000,1.595300e+05,159822.000000,161583.000000,161583.000000,161583.000000,158866.000000,159810.000000,159810.000000,...,128722.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000,17969.000000
mean,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,1.380283,44914.264783,20.161323,...,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973,39.149090
std,7.040855,6.259121,6.005746e+06,21941.656285,4.670207,6.750148,7.000456,0.750646,53413.302607,16.870350,...,12.329739,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143,10.503788
min,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,1.000000,1.000000,1.000000,...,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,1.000000,450.000000,8.000000,...,50.000000,26.000000,27.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000,31.000000
50%,66.000000,70.000000,7.250000e+05,4000.000000,25.000000,181.000000,75.000000,1.000000,1889.000000,17.000000,...,59.000000,50.000000,54.000000,52.000000,11.00000,11.000000,11.000000,11.000000,11.000000,41.000000
75%,70.000000,75.000000,1.800000e+06,10000.000000,28.000000,186.000000,80.000000,2.000000,110832.000000,27.000000,...,66.000000,63.000000,66.000000,64.000000,14.00000,14.000000,14.000000,14.000000,14.000000,46.000000
max,94.000000,95.000000,1.940000e+08,575000.000000,54.000000,208.000000,110.000000,5.000000,116361.000000,99.000000,...,96.000000,94.000000,94.000000,95.000000,91.00000,92.000000,95.000000,92.000000,94.000000,68.000000


Splitting categorical and quantitative data

In [ ]:
categorical = df.select_dtypes(include=['object'])

Exploring quantitative data

In [ ]:
quant_df = df.drop(columns = categorical.columns)

In [ ]:
quant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 58 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   overall                      161583 non-null  int64  
 1   potential                    161583 non-null  int64  
 2   value_eur                    159530 non-null  float64
 3   wage_eur                     159822 non-null  float64
 4   age                          161583 non-null  int64  
 5   height_cm                    161583 non-null  int64  
 6   weight_kg                    161583 non-null  int64  
 7   league_level                 158866 non-null  float64
 8   club_team_id                 159810 non-null  float64
 9   club_jersey_number           159810 non-null  float64
 10  nationality_id               161583 non-null  int64  
 11  nation_team_id               9316 non-null    float64
 12  nation_jersey_number         9316 non-null    float64
 13 

In [ ]:
corr_matrix = quant_df.corr()

In [ ]:
corr_matrix['overall'].sort_values(ascending=False)

overall                        1.000000
movement_reactions             0.845753
potential                      0.695362
mentality_composure            0.683307
passing                        0.681114
wage_eur                       0.611368
dribbling                      0.606575
release_clause_eur             0.578914
value_eur                      0.566827
physic                         0.513279
attacking_short_passing        0.499757
mentality_vision               0.493282
international_reputation       0.485234
skill_long_passing             0.483808
shooting                       0.482498
power_shot_power               0.480297
age                            0.458390
skill_ball_control             0.457011
skill_curve                    0.415863
power_long_shots               0.409086
mentality_aggression           0.398310
attacking_crossing             0.396833
skill_fk_accuracy              0.387269
goalkeeping_speed              0.385093
attacking_volleys              0.377048


**Exploring Categorical Data**

I think the important categorical features that could correlate strongly with the overall rating are body type, work rate and preferred foot

In [ ]:
important_categorical_features = [column for column in categorical if column in  ['body_type', 'work_rate', 'preferred_foot',]]

In [ ]:
non_important = [column for column in categorical.columns if column not in ['body_type',  'work_rate', 'preferred_foot', ]]

Adding columns that have a correlation score of less than 0.4 and greater than -0.4 to the non important columns list

In [ ]:
for column in quant_df.columns:
  if corr_matrix['overall'][column] < 0.4 and corr_matrix['overall'][column] > -0.4:
    non_important.append(column)
#add the values that don't correlate strongly to non important columns

In [ ]:
non_important

['player_url',
 'short_name',
 'long_name',
 'player_positions',
 'dob',
 'league_name',
 'club_name',
 'club_position',
 'club_loaned_from',
 'nationality_name',
 'nation_position',
 'real_face',
 'player_tags',
 'player_traits',
 'ls',
 'st',
 'rs',
 'lw',
 'lf',
 'cf',
 'rf',
 'rw',
 'lam',
 'cam',
 'ram',
 'lm',
 'lcm',
 'cm',
 'rcm',
 'rm',
 'lwb',
 'ldm',
 'cdm',
 'rdm',
 'rwb',
 'lb',
 'lcb',
 'cb',
 'rcb',
 'rb',
 'gk',
 'player_face_url',
 'height_cm',
 'weight_kg',
 'league_level',
 'club_team_id',
 'club_jersey_number',
 'nationality_id',
 'nation_team_id',
 'nation_jersey_number',
 'weak_foot',
 'skill_moves',
 'pace',
 'defending',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_volleys',
 'skill_dribbling',
 'skill_fk_accuracy',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_balance',
 'power_jumping',
 'power_stamina',
 'power_strength',
 'mentality_aggression',
 'mentality_interceptions',
 'm

In [ ]:
cat_pipeline =  OneHotEncoder(handle_unknown = 'ignore', sparse = False) #pipeline to handle categorical data

Dropping non important columns and columns with too many null values

In [ ]:
categorical.drop(columns = [column for column in categorical.columns if column in non_important], inplace = True)
categorical.drop(columns = [column for column in categorical.columns if categorical[column].isnull().sum() > 0.4*categorical.shape[0]], inplace = True)

quant_df.drop(columns = [column for column in quant_df.columns if column in non_important], inplace = True)
quant_df.drop(columns = [column for column in quant_df.columns if quant_df[column].isnull().sum() > 0.4*quant_df.shape[0]], inplace = True)


Encoding the categorical values

In [ ]:
categorical = cat_pipeline.fit_transform(categorical)
categorical = pd.DataFrame(categorical, columns = cat_pipeline.get_feature_names_out(important_categorical_features))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
quant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   overall                   161583 non-null  int64  
 1   potential                 161583 non-null  int64  
 2   value_eur                 159530 non-null  float64
 3   wage_eur                  159822 non-null  float64
 4   age                       161583 non-null  int64  
 5   international_reputation  161583 non-null  int64  
 6   release_clause_eur        103622 non-null  float64
 7   shooting                  143614 non-null  float64
 8   passing                   143614 non-null  float64
 9   dribbling                 143614 non-null  float64
 10  physic                    143614 non-null  float64
 11  attacking_short_passing   161583 non-null  int64  
 12  skill_curve               161583 non-null  int64  
 13  skill_long_passing        161583 non-null  i

Using the Iterative imputer to fill in the null values in some of the columns and then using the scaler to scale everything

In [ ]:
quant_cols = quant_df.columns
imputer = IterativeImputer(max_iter = 10, random_state = 0)
scaler = StandardScaler()
quant_df = imputer.fit_transform(quant_df)
quant_df = scaler.fit_transform(quant_df)
quant_df = pd.DataFrame(quant_df, columns = quant_cols)

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
quant_df.head()

,overall,potential,value_eur,wage_eur,age,international_reputation,release_clause_eur,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure
0,3.877514,3.875315,16.431396,24.663494,0.401872,10.148137,16.753480,2.418049,2.340534,2.469771,-0.315481,2.086204,2.306588,1.536683,2.281310,3.535775,1.499832,2.148684,2.588430,2.950846
1,3.735485,3.396013,12.833034,16.657776,0.830120,10.148137,12.913692,2.657693,1.979708,2.145853,1.199101,1.609554,2.251267,1.273977,2.039859,3.099571,2.401757,2.411698,1.957672,2.863904
2,3.451428,3.076478,8.732574,12.083080,1.044244,10.148137,8.859600,2.238316,2.124038,2.210637,-0.220820,1.881925,2.085305,1.536683,1.919133,2.990520,1.886371,2.253890,2.167924,2.709281
3,3.451428,3.076478,8.397843,12.083080,1.472491,10.148137,8.534713,2.537871,1.979708,1.821936,1.861730,1.745740,1.808701,1.536683,1.919133,2.554317,2.337334,2.148684,2.097840,2.907955
4,3.451428,3.076478,10.238865,13.226754,0.615996,10.148137,10.336124,-1.459028,-1.457565,-1.858994,3.000687,-1.114157,-1.233943,-0.761996,-1.642271,2.990520,-0.948252,-1.165296,-2.317466,0.589917


In [ ]:
quant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   overall                   161583 non-null  float64
 1   potential                 161583 non-null  float64
 2   value_eur                 161583 non-null  float64
 3   wage_eur                  161583 non-null  float64
 4   age                       161583 non-null  float64
 5   international_reputation  161583 non-null  float64
 6   release_clause_eur        161583 non-null  float64
 7   shooting                  161583 non-null  float64
 8   passing                   161583 non-null  float64
 9   dribbling                 161583 non-null  float64
 10  physic                    161583 non-null  float64
 11  attacking_short_passing   161583 non-null  float64
 12  skill_curve               161583 non-null  float64
 13  skill_long_passing        161583 non-null  f

Preparing the pipeline for future use with testing data

In [ ]:
#function for a pipeline
def drop_null_columns(df):
  for column in df.columns:
    if df[column].isnull().sum() > 0.4*df.shape[0]:
      df.drop(columns = [column])
  return df
drop_nulls = FunctionTransformer(drop_null_columns)

In [ ]:
#functions for a pipeline
def drop_columns(df, columns):
  return df.drop(columns = columns)
column_dropper = FunctionTransformer(drop_columns, kw_args = {'columns': non_important})

In [ ]:
quant = Pipeline([
    ('impute', imputer),
    ('scaler', scaler)
]) #pipeline for quantitative columns

column_transformer = ColumnTransformer(transformers = [
    ('cat_transformer', cat_pipeline, important_categorical_features), #this works on the important categorical features
    ('quant_transformer', quant, [column for column in df.columns if (df[column].dtype == 'float64' or df[column].dtype == 'int64') and column not in non_important])
    #works on quantitative features
]
)

pipeline = Pipeline([
    ('dropping_unimportant', column_dropper),
    ('dropping_null_columns', drop_nulls),
    ('preprocessing', column_transformer),

])
#X = pipeline.fit_transform(df) #transforms the dataframe according to the pipeline

In [ ]:
one_hot_features = cat_pipeline.get_feature_names_out(important_categorical_features) #gets the new features from the one hot encoding

In [ ]:
new_columns = list(one_hot_features)+ [column for column in df.columns if column not in non_important and column not in important_categorical_features]
new_columns

['preferred_foot_Left',
 'preferred_foot_Right',
 'work_rate_High/High',
 'work_rate_High/Low',
 'work_rate_High/Medium',
 'work_rate_Low/High',
 'work_rate_Low/Low',
 'work_rate_Low/Medium',
 'work_rate_Medium/High',
 'work_rate_Medium/Low',
 'work_rate_Medium/Medium',
 'body_type_Lean (170-)',
 'body_type_Lean (170-185)',
 'body_type_Lean (185+)',
 'body_type_Normal (170-)',
 'body_type_Normal (170-185)',
 'body_type_Normal (185+)',
 'body_type_Stocky (170-)',
 'body_type_Stocky (170-185)',
 'body_type_Stocky (185+)',
 'body_type_Unique',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'age',
 'international_reputation',
 'release_clause_eur',
 'shooting',
 'passing',
 'dribbling',
 'physic',
 'attacking_short_passing',
 'skill_curve',
 'skill_long_passing',
 'skill_ball_control',
 'movement_reactions',
 'power_shot_power',
 'power_long_shots',
 'mentality_vision',
 'mentality_composure']

Done preparing the pipeline.

Now training models

In [ ]:
X = pd.concat([categorical, quant_df], axis =1)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 41 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   preferred_foot_Left         161583 non-null  float64
 1   preferred_foot_Right        161583 non-null  float64
 2   work_rate_High/High         161583 non-null  float64
 3   work_rate_High/Low          161583 non-null  float64
 4   work_rate_High/Medium       161583 non-null  float64
 5   work_rate_Low/High          161583 non-null  float64
 6   work_rate_Low/Low           161583 non-null  float64
 7   work_rate_Low/Medium        161583 non-null  float64
 8   work_rate_Medium/High       161583 non-null  float64
 9   work_rate_Medium/Low        161583 non-null  float64
 10  work_rate_Medium/Medium     161583 non-null  float64
 11  body_type_Lean (170-)       161583 non-null  float64
 12  body_type_Lean (170-185)    161583 non-null  float64
 13  body_type_Lean

In [ ]:
X.head()

,preferred_foot_Left,preferred_foot_Right,work_rate_High/High,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,...,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.315481,2.086204,2.306588,1.536683,2.281310,3.535775,1.499832,2.148684,2.588430,2.950846
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.199101,1.609554,2.251267,1.273977,2.039859,3.099571,2.401757,2.411698,1.957672,2.863904
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.220820,1.881925,2.085305,1.536683,1.919133,2.990520,1.886371,2.253890,2.167924,2.709281
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.861730,1.745740,1.808701,1.536683,1.919133,2.554317,2.337334,2.148684,2.097840,2.907955
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.000687,-1.114157,-1.233943,-0.761996,-1.642271,2.990520,-0.948252,-1.165296,-2.317466,0.589917


In [ ]:

X.drop('overall', axis = 1, inplace = True)

Splitting data into the test data and training data

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = 0.2, random_state = 47)

In [ ]:
lin_reg_model = LinearRegression()

In [ ]:
lin_reg_model.fit(Xtrain, Ytrain)

LinearRegression()

In [ ]:
y_pred_1 = lin_reg_model.predict(Xtest)

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred_1, Ytest)},
Mean Squared Error = {mean_squared_error(y_pred_1, Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred_1, Ytest))},
R2 score = {r2_score(y_pred_1, Ytest)}
""")


Mean Absolute Error = 1.523367869646297,
Mean Squared Error = 3.8558583972599187,
Root Mean Squared Error = 1.9636339774153224,
R2 score = 0.915240677859888



Processing the test data (players_22)

Using the pipeline on the test data

In [ ]:
new_test_X = pipeline.fit_transform(players_22)
new_test_X = pd.DataFrame(new_test_X, columns = new_columns)
new_test_Y = players_22['overall']
new_test_X.drop('overall', axis=1, inplace = True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
new_test_X.head()

,preferred_foot_Left,preferred_foot_Right,work_rate_High/High,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,...,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.130135,2.216206,2.512940,2.522017,2.260399,3.554438,2.119026,2.447123,3.000047,3.137573
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.936889,1.598583,1.854710,1.601260,2.019401,3.664174,2.719163,2.395322,2.052781,3.055051
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.381482,1.735833,1.745006,1.140882,1.778402,3.444701,2.344077,1.980918,1.834181,2.477402
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.663646,1.941707,2.238678,1.864334,2.200150,3.225227,1.668922,1.929117,2.635714,2.890009
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.025807,2.422080,2.074120,2.653554,2.019401,3.225227,2.494112,2.291721,2.927180,2.724966


Testing the linear regression on the test data

In [ ]:
new_test_pred = lin_reg_model.predict(new_test_X)

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(new_test_pred, new_test_Y)},
Mean Squared Error = {mean_squared_error(new_test_pred, new_test_Y)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(new_test_pred, new_test_Y))},
R2 score = {r2_score(new_test_pred, new_test_Y)}
""")


Mean Absolute Error = 1.5359058766751676,
Mean Squared Error = 3.971592594524908,
Root Mean Squared Error = 1.9928854945844,
R2 score = 0.9090558617480233



Experimenting with a decision tree

In [ ]:
dTree = DecisionTreeRegressor()

In [ ]:
dTree.fit(Xtrain, Ytrain)

DecisionTreeRegressor()

In [ ]:
tree_pred = dTree.predict(Xtest)

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(tree_pred, Ytest)},
Mean Squared Error = {mean_squared_error(tree_pred, Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(tree_pred, Ytest))},
R2 score = {r2_score(tree_pred, Ytest)}
""")


Mean Absolute Error = 0.6530618559891079,
Mean Squared Error = 1.6013553238233746,
Root Mean Squared Error = 1.265446689443445,
R2 score = 0.9677732356426371



Using the test data on the decision tree

In [ ]:
new_tree_pred = dTree.predict(new_test_X)

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(new_tree_pred, new_test_Y)},
Mean Squared Error = {mean_squared_error(new_tree_pred, new_test_Y)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(new_tree_pred, new_test_Y))},
R2 score = {r2_score(new_tree_pred, new_test_Y)}
""")


Mean Absolute Error = 0.9527027027027027,
Mean Squared Error = 2.414168074324324,
Root Mean Squared Error = 1.553759336037703,
R2 score = 0.9486360100206243



Training an ensemble model. The Random Forest Regressor

In [ ]:
forest_reg = RandomForestRegressor(n_jobs = -1) #this is an ensemble model

In [ ]:
parameters = {
    'n_estimators': [10,50,100],
    'max_depth': [5,10, 15, 20]
}

In [ ]:
rand_search = RandomizedSearchCV(forest_reg, parameters, scoring='neg_root_mean_squared_error', cv = 3, n_jobs=-1)
 #grid search helps us figure out the best hyper parameters based on the parameters model
 #but randomized search is faster

Training with the training data

In [ ]:
rand_search.fit(Xtrain, Ytrain)

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1), n_jobs=-1,
                   param_distributions={'max_depth': [5, 10, 15, 20],
                                        'n_estimators': [10, 50, 100]},
                   scoring='neg_root_mean_squared_error')

Initial testing

In [ ]:
rand_pred = rand_search.predict(Xtest)

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(rand_pred, Ytest)},
Mean Squared Error = {mean_squared_error(rand_pred, Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(rand_pred, Ytest))},
R2 score = {r2_score(rand_pred, Ytest)}
""")


Mean Absolute Error = 0.5144869823931079,
Mean Squared Error = 0.7030168157979192,
Root Mean Squared Error = 0.8384609804862234,
R2 score = 0.9855058282513386



Testing with the players 22 data

In [ ]:
rand_pred_new = rand_search.predict(new_test_X)

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(rand_pred_new, new_test_Y)},
Mean Squared Error = {mean_squared_error(rand_pred_new, new_test_Y)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(rand_pred_new, new_test_Y))},
R2 score = {r2_score(rand_pred_new, new_test_Y)}
""")


Mean Absolute Error = 0.7300789802141737,
Mean Squared Error = 1.2433756853191378,
Root Mean Squared Error = 1.1150675698445982,
R2 score = 0.9729339320292765



Investigating feature importances.
We currently have 40 features.
Too much for a user to enter.

In [ ]:
feature_importances = pd.DataFrame({'features': X.columns, 'importances': rand_search.best_estimator_.feature_importances_})

In [ ]:
feature_importances.sort_values(by='importances', ascending=False)

,features,importances
22,value_eur,0.801374
24,age,0.084620
21,potential,0.042795
35,movement_reactions,0.020173
23,wage_eur,0.017171
26,release_clause_eur,0.015177
30,physic,0.003948
29,dribbling,0.002243
34,skill_ball_control,0.001726
39,mentality_composure,0.001388


We can see from above that the features that are most important to our random forest regressor are potential, value_eur, age, release_clause_eur and movement_reactions

We split our training and test data based on these features.
We are creating training and test data that only has these features. Note that this is from data we used to train only.

In [ ]:
features = ['potential', 'value_eur','wage_eur', 'age', 'release_clause_eur', 'movement_reactions']
Xtrain_sub = pd.concat([Xtrain[feature] for feature in features], axis = 1)
Xtest_sub = pd.concat([Xtest[feature] for feature in features], axis =1)
Xtrain_sub.head()

,potential,value_eur,wage_eur,age,release_clause_eur,movement_reactions
93808,0.040899,-0.225674,-0.131358,0.187748,-0.200331,-0.062903
43724,0.200667,-0.296805,-0.268599,-0.454624,-0.287634,-0.826259
146791,0.040899,-0.070861,0.326111,0.830120,-0.006667,0.591402
93405,0.040899,-0.204753,-0.131358,0.187748,-0.200331,0.373300
89746,0.200667,-0.104334,-0.085611,1.686615,-0.163443,0.809504


Creating a new model that based on what our random search cross validation found and training the model

In [ ]:
new_model = rand_search = RandomizedSearchCV(forest_reg, parameters, scoring='neg_root_mean_squared_error', cv = 3, n_jobs=-1)
new_model.fit(Xtrain_sub, Ytrain)

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1), n_jobs=-1,
                   param_distributions={'max_depth': [5, 10, 15, 20],
                                        'n_estimators': [10, 50, 100]},
                   scoring='neg_root_mean_squared_error')

Initial testing on our new model

In [ ]:
new_model_pred= new_model.predict(Xtest_sub)

The model does quite well even though we've reduced data

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(new_model_pred, Ytest)},
Mean Squared Error = {mean_squared_error(new_model_pred, Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(new_model_pred, Ytest))},
R2 score = {r2_score(new_model_pred, Ytest)}
""")


Mean Absolute Error = 0.5682362081902999,
Mean Squared Error = 0.9220569944955996,
Root Mean Squared Error = 0.960237988467234,
R2 score = 0.9810327411304409



Testing with the players 22 data

We need to create a version of the players 22 data that only has the key features. We used the pd.concat function to create this subset

In [ ]:
new_model_pred_new_data = new_model.predict(pd.concat([new_test_X[feature] for feature in features], axis = 1))

The model is slightly worse, but its decent

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(new_model_pred_new_data, new_test_Y)},
Mean Squared Error = {mean_squared_error(new_model_pred_new_data, new_test_Y)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(new_model_pred_new_data, new_test_Y))},
R2 score = {r2_score(new_model_pred_new_data, new_test_Y)}
""")


Mean Absolute Error = 0.8643553087978052,
Mean Squared Error = 1.7869406024941765,
Root Mean Squared Error = 1.3367649765363305,
R2 score = 0.9602495742464303



In [ ]:
scaler = StandardScaler()
scaler.fit(df[features]) #fit the scaler on the reduced feature set from the legacy training set
model_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', new_model)
])


Saving the pipeline and the model

In [ ]:
pkl.dump(model_pipeline, open('/content/model_pipeline.pkl', 'wb'))